Create an electronic report in English with a maximum of 2000 words (excluding citations) using Jupyter. The report should include the posed question, conducted analysis, and derived conclusion. Only one team member needs to submit this report. It is not required to include all tasks completed by every group member in their individual assignments; tailor the final report to the collective group's work. 

You must submit 2 files: an .html file (File -> Download As -> HTML) an .ipynb file. This file must be fully reproducible. It must run completely from top to bottom without any additional files.

**Title Introduction:**
provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report
clearly state the question you tried to answer with your project
identify and fully describe the dataset that was used to answer the question

**Methods & Results:**
describe the methods you used to perform your analysis from beginning to end that narrates the analysis code.
your report should include code which:
loads data 
wrangles and cleans the data to the format necessary for the planned analysis
performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
performs the data analysis
creates a visualization of the analysis 
note: all figures should have a figure number and a legend

**Discussion:**
summarize what you found
discuss whether this is what you expected to find?
discuss what impact could such findings have?
discuss what future questions could this lead to?

**References:**
You may include references if necessary, as long as they all have a consistent citation style.

<h2 style="font-size: xxx-large;">Group 21 Project.</h2>

<h4 style="font-size: xx-large;">Introduction</h4>

<h4 style="font-size: x-medium;">Introduction to our question:</h4>
For this project, our group chose to investigate question 3 on demand forcasting, namely - "what time windows are most likely to have large number of simultaneous players", to ensure the server has a sufficient number of licences in order to keep up with concurrent players. Reflecting the courses philosphy of transform -> visualize -> model -> repeat, and our focus on modeling over the last few weeks, we were specifically interested in whether we could use the preexisting server data to predict what times may have the most players, and the probably upper bounds of player activity we could reasonably predict, in order to advise the server team if they need to increase their server capacity.
<br>
<h4 style="font-size: x-medium;">The data:</h4>
In order to begin our investigation, we were given 2 dataframes, one on player information and one on play sessions information, both of which we deemed important for us to answer our question:

The sessions dataframe includes 1606 rows, with each corresponding to a single play session, and 5 columns, titled:
- start_time - The time and date that a player logged onto the server and began to play, as a string in a DD/MM/YYYY format for data and a 24 hour time.
- end_time - The time and date that a player logged off the server after stopping playing or being kicked off, as a string in a DD/MM/YYYY format for data and a 24 hour time.
- original_start_time - The same information as start_time but measured in unix time, as a float which measures from Jan 1st, 1970 (Wikipedia)
- original_end_time - The same information as end_time but as a float measured in unix time 
- hashedEmail - A string identifier that attributed each play session to a specific individual

This granular data on the dates of player logins and their session start/end times is useful to our investivation as it allowed us to analyze overlap between their playtimes and find trends in playtime.
<br>
<br>
The players dataframe includes information on each player, having 196 rows which corresponded to a player each, and 7 columns, titled:
- experience - A metric of self-reported experience within the game, an ordinal with categories 'amateur', 'beginner', 'regular', 'pro', and 'veteran'
- subscribe - Whether the player has or has not subscribed to the servers game-related newsletter
- hashedEmail - A string identifier that attributed each play session to a specific individual, this is the same 'hashedEmail' as in the sessions dataset, which allowed for merging both on this variable
- played_hours - The number of hours each player cumulatively put into playing on the minecraft server, reported by the player and saved as a float (being a number with a decimal). We changed it to lifetime_hours in the merged dataframe
- name - The player's name, reported by the player, saved as a string
- age - The player's age, reported by the player, saved as an integer value
- individualID - NAN values
- organizationname - NAN values

This data is useful as it enabled us to attribute our findings about playtime/play sessions to specific individuals, and can help in investigating further into trends in play sessions.
<h4 style="font-size: x-medium;">Issues:</h4>
In our cross-analysis of dataframes, we came across 3 main issues we thought were valuable to mention:

1.  First, we noticed that there were some hashedEmails not attributed to any playtimes, which means that there are some players who registered who have never played, and therefore are not tracked in the sessions frame. 
2.  The second issue we noticed was in the self-reporting process of personal information. While logistically impossible to collect data otherwise, self reporting can lead to people lying or putting false information, meaning that our information about played hours or experience could be faulty, and even throw our analysis entirely off.
3. The final issue is the relative measure of experience, relying on the individuals own interpretation of what a beginner/amatuer/veteran/pro is.

These issues didn't end up influencing our results heavily, however, as our question centered around play sessions, not necessarily the players behind them, which means we could drop any player information that was attributed to people who never logged into the server, and could ignore most of the self-reported information that was possibly faulty.

<h4 style="font-size: xx-large;">Methods & Results</h4>
(describe the methods you used to perform your analysis from beginning to end that narrates the analysis code. your report should include code which: loads data wrangles and cleans the data to the format necessary for the planned analysis performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis performs the data analysis creates a visualization of the analysis note: all figures should have a figure number and a legend)
<br>
<br>
<h4 style="font-size: x-medium;">Importing/tidying:</h4>
For our investigation, our method to address question 3 incorporated a regression model to predict the activity/concurrent players vs the time of day/day of the week, then comparing them to specific categories of days like weekdays/weekends/holidays to find spikes in player activity. We thought regression would be appropriate because the goal of regression is to predict a numerical value, which is applicable for the numerical measurements of time in the sessions dataset.
<br>
<br>
Our preparation included:

1. Importing all relevant packages and functions
2. Importing our dataframes and tidying them by dropping NAN columns
3. Merging dataframes on hashedEmail

In [30]:
#Importing relevant packages/functions
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Simplify working with large datasets in Altair
alt.data_transformers.enable('vegafusion')

# Output dataframes instead of arrays
#set_config(transform_output="pandas")
# this can break the regression model earlier because it forces a sparse matrix because sklearn can't work with dataframes for OneHotEncoder() into LinearRegression()

DataTransformerRegistry.enable('vegafusion')

In [31]:
#Merge Dataframes for simplicity ***im not entirely sure we need this, but keep for now

url_players = "https://drive.google.com/uc?id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz"
players = pd.read_csv(url_players).drop(columns = ["individualId", "organizationName"])

url_sessions = "https://drive.google.com/uc?id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB"
sessions = pd.read_csv(url_sessions).drop(columns=['original_start_time','original_end_time'])

players_sessions = sessions.merge(players, on = 'hashedEmail')
#players_sessions

<h4 style="font-size: x-medium;">Wrangling</h4>
To prepare our data for our model specifically, we:

1. Converted time in start_time and end_time from strings to datetime objects
2. Counted the amount of session playtime by each unique user

In [32]:
#Converting time strings to datetime objects. Notice the slight change in formatting 
players_sessions["start_time"] = pd.to_datetime(players_sessions["start_time"], dayfirst=True)
players_sessions["end_time"] = pd.to_datetime(players_sessions["end_time"], dayfirst=True)
players_sessions = players_sessions.rename(columns={"played_hours" : "lifetime_hours"})
players_sessions.head()

,hashedEmail,start_time,end_time,experience,subscribe,lifetime_hours,name,gender,age
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,2024-06-30 18:12:00,2024-06-30 18:24:00,Regular,True,223.1,Hiroshi,Male,17
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,2024-06-17 23:33:00,2024-06-17 23:46:00,Amateur,True,53.9,Alex,Male,17
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,2024-07-25 17:34:00,2024-07-25 17:57:00,Amateur,True,150.0,Delara,Female,16
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,2024-07-25 03:22:00,2024-07-25 03:58:00,Regular,True,223.1,Hiroshi,Male,17
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,2024-05-25 16:01:00,2024-05-25 16:12:00,Amateur,True,53.9,Alex,Male,17


In [33]:
#temp
players_sessions.to_csv("unsorted_players_sessions.csv")

In [34]:
#Inputting timedelta64 objects from end time and start time with pandas tools
timedelta = players_sessions["end_time"] - players_sessions["start_time"]
players_sessions["session_length_minutes"] = timedelta.dt.total_seconds()/60
players_sessions_timedelta = players_sessions.copy(deep=True)
players_sessions_timedelta["timedelta"] = timedelta

timedelta
#currently IS USED

0      0 days 00:12:00
1      0 days 00:13:00
2      0 days 00:23:00
3      0 days 00:36:00
4      0 days 00:11:00
             ...      
1530   0 days 00:06:00
1531   0 days 00:11:00
1532   0 days 00:21:00
1533   0 days 00:07:00
1534   0 days 00:19:00
Length: 1535, dtype: timedelta64[ns]

In [35]:
#REMOVE LATER

#does this do anything useful or is this actually working against us, didn't merging alreading 
#sessions_dropped = players_sessions.drop(players_sessions[players_sessions['lifetime_hours'] == 0].index)
#players_sessions[players_sessions['session_length_minutes'] == 0].head(40)#yeah... we already have no 0 session time observations logged, I'm just wondering how this happened? As the player dataframe has a few players who have no playtime
#sessions_dropped.head()

In [36]:
#REMOVE IN FINAL ANALYSIS (does not modify any dfs)
players_sessions[players_sessions['lifetime_hours'] == 0.1].sort_values(by="lifetime_hours").head()

,hashedEmail,start_time,end_time,experience,subscribe,lifetime_hours,name,gender,age,session_length_minutes
46,6b1cdc07fcc1f7ea09509341fd245dd34fdba386f14a49...,2024-06-25 22:58:00,2024-06-25 23:09:00,Veteran,True,0.1,Finnian,Non-binary,17,11.0
70,4bfad3613c71ace05644bf210195d9fb0d3d9513753ad2...,2024-07-28 08:49:00,2024-07-28 09:00:00,Veteran,False,0.1,Rocco,Male,17,11.0
89,8b71f4d66a38389b7528bb38ba6eb71157733df7d17403...,2024-04-16 05:09:00,2024-04-16 05:22:00,Amateur,True,0.1,Natalie,Male,17,13.0
103,dc73467f73263dd4a07838330dd1cc115aa3f8b0353891...,2024-05-25 01:36:00,2024-05-25 01:43:00,Veteran,True,0.1,Felix,Male,21,7.0
155,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,2024-04-25 02:04:00,2024-04-25 02:13:00,Regular,True,0.1,Kylie,Male,21,9.0


The following is processing the players_sessions data into time-series data, and tracking the concurrent players for each interval (intervals of 1h).

First an empty "timeline" dataframe is created with intervals of 10 minutes, between Apr 6-Sept 26, 2025.
And then using series-value comparisons (to return a series of T/F), the amount of players online during each interval is found (so the entire sessions dataframe is checked for each entry in timeline).

Unfortunately because of the comparisons between the loop index's timestamp and start_time and end_time series', if a session took place entirely within two timestamps without "crossing" them (e.g. if hourly, then not crossing 6pm or 7pm sharp), the session would not be logged

Note that a nested for loop which may be more intuitive took too long to be practical, and was scrapped. Also note that "players_sessions" and "sessions" may be used interchangeably to refer to each other.

In [37]:
#players_sessions["start_time"].max()

In [105]:
timeline = pd.date_range(
    pd.Timestamp("2024-04-06 09:20:00"), #1st instance is apr 6 09:27:00
    pd.Timestamp("2024-09-26 09:15:00"), #last instance is sept 26, 06:09:00
    freq="5min",
    name="timestamp"
)
timeline = timeline.to_frame().reset_index(drop=True)
timeline["concurrent"] = 0
timeline
#Note: Length is 49824 rows. There are 173 days between Apr 6 to Sept 26. 173 * 24 * 12 = 49824

,timestamp,concurrent
0,2024-04-06 09:20:00,0
1,2024-04-06 09:25:00,0
2,2024-04-06 09:30:00,0
3,2024-04-06 09:35:00,0
4,2024-04-06 09:40:00,0
...,...,...
49819,2024-09-26 08:55:00,0
49820,2024-09-26 09:00:00,0
49821,2024-09-26 09:05:00,0
49822,2024-09-26 09:10:00,0


In [106]:
#np.array([True, False, True]).sum()

In [109]:
concurrent_counts = []

for snapshot in timeline.index:
    sessions_active_now = (
        (timeline["timestamp"][snapshot] >= players_sessions['start_time'])
        & 
        (timeline["timestamp"][snapshot] < players_sessions['end_time'] ) 
        #so comparing against the snapshot window, make a Series same length as sessions.csv where True means a player is online in that window. This is the boolean AND of the set of True from timestamp >= session start time (series!), and timestamp < session end time (also series!) Note that this is always 8 True or under, so it's slim pickings   
    ) #this produces a Series of trues and falses (mostly False), 1535 long, and we will do this for every snapshot (~4000)

    number_active = sessions_active_now.sum() #sum counts every True as 1, every False as 0. this is the amount of sessions active now for a given timestamp
    concurrent_counts.append(number_active) #this is a List object which 
##END OF FOR LOOP


In [110]:
#This represents the amount of players that were concurrently active during each timestamp. 4152 timestamps. It sums to 4152.
pd.Series(concurrent_counts).value_counts()

0    38996
1     7446
2     2402
3      705
4      212
5       28
7       19
6       15
8        1
Name: count, dtype: int64

In [111]:
players_sessions.sort_values(by="start_time").head()

,hashedEmail,start_time,end_time,experience,subscribe,lifetime_hours,name,gender,age,session_length_minutes
1050,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,2024-04-06 09:27:00,2024-04-06 09:31:00,Amateur,True,56.1,Dana,Male,23,4.0
894,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,2024-04-06 09:35:00,2024-04-06 10:16:00,Amateur,True,56.1,Dana,Male,23,41.0
1247,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,2024-04-06 20:56:00,2024-04-06 22:04:00,Amateur,True,56.1,Dana,Male,23,68.0
417,f6daba428a5e19a3d47574858c13550499be23603422e6...,2024-04-06 22:24:00,2024-04-06 23:33:00,Pro,True,30.3,Morgan,Male,9,69.0
94,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,2024-04-07 00:17:00,2024-04-07 00:28:00,Amateur,True,56.1,Dana,Male,23,11.0


In [112]:
timeline = timeline.assign(concurrent=concurrent_counts)
timeline

,timestamp,concurrent
0,2024-04-06 09:20:00,0
1,2024-04-06 09:25:00,0
2,2024-04-06 09:30:00,1
3,2024-04-06 09:35:00,1
4,2024-04-06 09:40:00,1
...,...,...
49819,2024-09-26 08:55:00,0
49820,2024-09-26 09:00:00,0
49821,2024-09-26 09:05:00,0
49822,2024-09-26 09:10:00,0


In [113]:
timeline["hour"] = timeline["timestamp"].dt.hour
timeline['day'] = timeline['timestamp'].dt.day_name()
timeline

,timestamp,concurrent,hour,day
0,2024-04-06 09:20:00,0,9,Saturday
1,2024-04-06 09:25:00,0,9,Saturday
2,2024-04-06 09:30:00,1,9,Saturday
3,2024-04-06 09:35:00,1,9,Saturday
4,2024-04-06 09:40:00,1,9,Saturday
...,...,...,...,...
49819,2024-09-26 08:55:00,0,8,Thursday
49820,2024-09-26 09:00:00,0,9,Thursday
49821,2024-09-26 09:05:00,0,9,Thursday
49822,2024-09-26 09:10:00,0,9,Thursday


<h4 style="font-size: xx-large;">Discussion</h4>
(Summarize what you found discuss whether this is what you expected to find? discuss what impact could such findings have? discuss what future questions could this lead to?)
<br>
<br>
While quite a simple analysis on a rudimentary dataset like the one provided, we believe that 

<h4 style="font-size: xx-large;">References (Cited ____)</h4> 
(You may include references if necessary, as long as they all have a consistent citation style)
<br>
<br>
https://en.wikipedia.org/wiki/Unix_time
https://stackoverflow.com/questions/56611698/pandas-how-to-read-csv-file-from-google-drive-public
